## Import

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
#import matplotlib as mpl
import psycopg2
import pandas.io.sql as sqlio

import matplotlib 
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import seaborn as sns

from textwrap import wrap

## Datenbankverbindung

In [2]:
con_osm = psycopg2.connect(database="osm", user="postgres", password="postgres", host="ma_db_01", port="5432")
con_postgres = psycopg2.connect(database="postgres", user="postgres", password="postgres", host="ma_db_01", port="5432")
#autocommit mode - true
con_osm.autocommit = True
con_postgres.autocommit = True

## Modell Grünheit

Daten aus PostGIS DB laden

gemäß dem Modell die entsprechenden vier INPUT und eine OUTPUT Variable

In [3]:
sql = "SELECT ogc_fid, in_1, in_2, in_3, in_4, out FROM nutzung_flurstueck_bbox;"

df_1 = sqlio.read_sql_query(sql, con_osm)
df_1.head(20)

,ogc_fid,in_1,in_2,in_3,in_4,out
0,41304,0,0,0,0,1
1,9475,0,0,0,0,0
2,9510,0,0,0,0,0
3,9511,0,0,0,0,0
4,19970,0,0,0,0,0
5,41389,0,0,0,0,0
6,45092,37,0,1,19,1
7,35860,0,0,0,0,0
8,47676,0,0,0,0,1
9,16395,0,0,7,0,0


Daten in "data" (Input-Variablen) und "target" (Output) aufteilen

In [75]:
data = df_1[['in_1', 'in_2', 'in_3', 'in_4']]
data.head()

,in_1,in_2,in_3,in_4
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


In [76]:
target = df_1[['out']]
target.head()

,out
0,1
1,0
2,0
3,0
4,0


## Logistic Regression

In [28]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

clf.fit(data, target.values.ravel())

LogisticRegression()

In [29]:
clf.score(data, target.values.ravel())

0.9226915132070497

## Support Vector Classifier (SVC)

In [39]:
from sklearn.svm import SVC
model1 = SVC(kernel='linear', C=1)
model1.fit(data, target.values.ravel())
model1.score(data, target.values.ravel())

0.9222959609721795

## nicht-lineare Support-Vector Machine / Radial Basis

In [9]:
from sklearn.svm import SVC
model1 = SVC(kernel='rbf', C=1E3, gamma=1)
model1.fit(data, target.values.ravel())
model1.score(data, target.values.ravel())

0.9247571748780381

## Aufteilen in Trainingsdaten und Testdaten

In [77]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target.values.ravel(), 
                                                    test_size=0.30, 
                                                    random_state=42)


In [78]:
X_train

,in_1,in_2,in_3,in_4
16612,0,0,0,0
9655,0,0,0,0
11406,0,0,0,0
11909,0,0,0,0
19667,0,0,0,0
...,...,...,...,...
11964,0,0,0,0
21575,0,0,0,0
5390,0,0,0,0
860,0,0,0,0


In [79]:
y_train

array([0, 0, 0, ..., 0, 0, 0])

In [73]:
from sklearn.svm import SVC
model1 = SVC(kernel='linear', C=1)
model1.fit(X_train, y_train)
model1.score(X_test, y_test)

0.9254321711104601

In [85]:
from sklearn.svm import SVC
model1 = SVC(kernel='rbf', C=1E3, gamma=0.1)
model1.fit(X_train, y_train)
model1.score(X_test, y_test)

0.9245531790213888